In [1]:
# Create reducer
from learning_observer.stream_analytics.helpers import kvs_pipeline, KeyField, Scope

@kvs_pipeline(scope=Scope([KeyField.STUDENT]), module_override='testing')
async def event_counter(event, state):
    if state is None:
        state = {}
    state['event_count'] = state.get('event_count', 0) + 1
    return state, state

In [2]:
import learning_observer.jupyter_helpers

ID = 'event_counter'
module = 'example_mod'

# add endpoint
learning_observer.jupyter_helpers.add_reducer_to_execution_dag(ID, event_counter, module, {'event_count': 0})
# start communication protocol endpoint
# await learning_observer.jupyter_helpers.serve_communication_protocol_endpoint()

Loading execution DAG from example_mod


In [3]:
# start communication protocol endpoint
await learning_observer.jupyter_helpers.serve_communication_protocol_endpoint()

Button(description='Start Server', style=ButtonStyle())

Button(description='Stop Server', disabled=True, style=ButtonStyle())

In [5]:
# Create a dashboard to connect to the reducer you just wrote
# This basic dashbaord will show the message back from the server
# in a markdown format.
import dash
from dash import Dash, html, dcc, callback, Output, Input, State, clientside_callback, Patch
import datetime
import json
import lo_dash_react_components as lodrc
import plotly.graph_objects as go

app = Dash(__name__)

fig = go.Figure()

app.layout = html.Div([
    html.H4('Graph of event count'),
    dcc.Graph(id='graph', figure=fig),
    html.H4('Incoming data.'),
    dcc.Markdown(id='out'),
    lodrc.LOConnection(id='ws', url='ws://localhost:8765/ws')
])

clientside_callback(
    '''function(msg) {
        console.log(msg)
        if (!msg) {
            return window.dash_clientside.no_update;
        }
        // TODO extract the event count and then add it to a graph that also does not yet exist
        return `\`\`\`json\n${JSON.stringify(JSON.parse(msg.data), null, 2)}\n\`\`\``; }''',
    Output('out', 'children'),
    Input('ws', 'message')
)
# Send connection information on the websocket when the connected
clientside_callback(
    f'''function(state) {{
        if (state === undefined) {{
            return window.dash_clientside.no_update;
        }}
        if (state.readyState === 1) {{
            return JSON.stringify({{"test": {{"execution_dag": "{module}", "target_exports": ["event_count"], "kwargs": {{"course_id": 12345}}}}}});
        }}
    }}''',
    Output('ws', 'send'),
    Input('ws', 'state')
)

@callback(
    Output('graph', 'figure'),
    Input('ws', 'message')
    # NOTE it is best practice to use a clientside
    # callback for websockets. Regular callbacks
    # will make more network requests than needed.
    # We use a regular callback here to demonstrate
    # some graphing capabilities.
)
def update_graph(msg):
    if msg is None:
        raise dash.exceptions.PreventUpdate
    now = datetime.datetime.now()
    data_raw = msg.get('data', {})
    students = json.loads(data_raw).get('test', {}).get('event_count', [])
    if len(students) == 0:
        raise dash.exceptions.PreventUpdate
    count = students[0].get('event_count', {})
    p = Patch()
    p['data'][0]['x'].append(now)
    p['data'][0]['y'].append(count)
    return p


app.run_server(jupyter_mode='inline', debug=True)

In [8]:
# Run reducer over events this can be done with a list events or a sample file
import learning_observer.offline

# list of events
events = [{}] * 3
await learning_observer.offline.process_file(events_list=events, source="localhost.testcase", pipeline=event_counter, userid='Tester')

# sample file
import os
sample_file = os.path.join('learning_observer', 'logs', 'sample01.log')
await learning_observer.offline.process_file(file_path=sample_file, source="localhost.testcase", pipeline=event_counter, userid='Tester')

<function kvs_pipeline.<locals>.decorator.<locals>.wrapper_closure.<locals>.process_event at 0x7f2bcc484d30>
<function kvs_pipeline.<locals>.decorator.<locals>.wrapper_closure.<locals>.process_event at 0x7f2bcc3edc60>


(1214, 'localhost.testcase', 'Tester')